In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# Cài đặt thư viện

In [ ]:
!pip install -q transformers datasets underthesea scikit-learn
!pip install -q --upgrade torchvision
!pip install -q underthesea
!pip install -q datasets tensorflow
!pip install -q --upgrade transformers

# Import thư viện

In [ ]:
import pandas as pd
import numpy as np
import torch
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB


import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional

import os
os.environ["WANDB_DISABLED"] = "true"

# Tải dataset lên dataframe

In [ ]:
try:
    df = pd.read_csv('/content/drive/MyDrive/XayDungMoHinh/momo_review_clean_balanced.csv')
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'momo_reviews_balanced.csv'.")
    # Dừng thực thi nếu không có file
    exit()


# Giữ lại các cột cần thiết

In [ ]:
df = df[['Noi_dung_sach_giu_dau', 'Noi_dung_sach', 'Nhãn']]
df = df.dropna().reset_index(drop=True)

# Tạo các label và hàm tách từ

In [ ]:
label_map = {"Tiêu cực": 0, "Trung lập": 1, "Tích cực": 2}
df['label'] = df['Nhãn'].map(label_map)

id_to_label = {v: k for k, v in label_map.items()}

# Hàm tách từ cho PhoBERT
def word_segment(text):
    """Tách từ tiếng Việt và nối các âm tiết bằng dấu gạch dưới."""
    return word_tokenize(text, format="text")

# Áp dụng tách từ cho cả cột có dấu và không dấu
# Chúng ta sẽ dùng cột `text` làm đầu vào chung cho mô hình sau này
df['text_accented_segmented'] = df['Noi_dung_sach_giu_dau'].apply(word_segment)
df['text_unaccented_segmented'] = df['Noi_dung_sach'].apply(word_segment)

# Chia các tập train, test và tăng cường dữ liệu

In [ ]:
# Chọn các cột cần thiết để chia
data_to_split = df[['text_accented_segmented', 'text_unaccented_segmented', 'label']]

# Chia dữ liệu (80% train, 20% còn lại) - stratify để giữ cân bằng nhãn
train_df, temp_df = train_test_split(
    data_to_split,
    test_size=0.2,
    random_state=42,
    stratify=data_to_split['label']
)

# Chia 20% còn lại thành validation và test (mỗi tập 10%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['label']
)

# Tăng cường dữ liệu cho tập train
train_accented = train_df[['text_accented_segmented', 'label']].rename(columns={'text_accented_segmented': 'text'})
train_unaccented = train_df[['text_unaccented_segmented', 'label']].rename(columns={'text_unaccented_segmented': 'text'})

train_aug_df = pd.concat([train_accented, train_unaccented]).sample(frac=1).reset_index(drop=True)

print(f"Kích thước tập Train (chưa tăng cường): {len(train_df)}")
print(f"Kích thước tập Train (đã tăng cường): {len(train_aug_df)}")
print(f"Kích thước tập Validation: {len(val_df)}")
print(f"Kích thước tập Test: {len(test_df)}")


Kích thước tập Train (chưa tăng cường): 17444
Kích thước tập Train (đã tăng cường): 34888
Kích thước tập Validation: 2181
Kích thước tập Test: 2181


# Mô hình PhoBERT

## Tokenize dữ liệu

In [ ]:
# Tải tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Chuyển đổi pandas DataFrames thành Hugging Face Datasets
# Tập train sử dụng dữ liệu đã tăng cường
train_dataset = Dataset.from_pandas(train_aug_df)

# Tập val và test chỉ dùng văn bản có dấu để đánh giá thực tế
val_dataset = Dataset.from_pandas(val_df[['text_accented_segmented', 'label']].rename(columns={'text_accented_segmented': 'text'}))
test_dataset = Dataset.from_pandas(test_df[['text_accented_segmented', 'label']].rename(columns={'text_accented_segmented': 'text'}))


# Hàm để tokenize dữ liệu
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

# Áp dụng tokenization cho các tập dữ liệu
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Xóa các cột không cần thiết
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["text"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["text"])

print("\nCấu trúc một mẫu dữ liệu sau khi tokenize:")
print(tokenized_train_dataset[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/34888 [00:00<?, ? examples/s]

Map:   0%|          | 0/2181 [00:00<?, ? examples/s]

Map:   0%|          | 0/2181 [00:00<?, ? examples/s]


Cấu trúc một mẫu dữ liệu sau khi tokenize:
{'label': 0, 'input_ids': [0, 59162, 1236, 6252, 266, 811, 21832, 14760, 105, 39245, 5109, 1656, 2801, 1517, 2006, 43158, 10081, 1656, 2644, 4145, 17662, 2008, 11855, 1187, 39433, 5791, 31692, 2006, 201, 19936, 7249, 1820, 82, 82, 305, 76, 107, 266, 4616, 3014, 2662, 7220, 17662, 266, 5301, 2049, 22785, 2052, 2033, 63353, 8738, 5642, 277, 55230, 50752, 201, 48596, 17662, 3240, 3069, 266, 18111, 47626, 305, 13086, 5301, 2049, 26308, 5241, 10242, 22899, 2801, 4907, 5255, 2667, 41106, 1881, 22899, 26, 305, 10249, 2006, 4616, 1236, 13488, 266, 91, 23557, 3907, 4129, 9375, 12728, 188, 3657, 13181, 3014, 2662, 9023, 10042, 6588, 13373, 2317, 2667, 2662, 138, 2292, 24805, 1204, 31114, 12561, 2008, 3587, 1993, 2801, 1510, 1340, 4129, 266, 4616, 3014, 2662, 3907, 4129, 51428, 3173, 14171, 3831, 52018, 2644, 8738, 5344, 25837, 11055, 32234, 6190, 1672, 11897, 7274, 2081, 4129, 2662, 2667, 41106, 2052, 105, 39245, 5109, 1656, 13568, 22975, 9901, 305, 26

## Khởi tạo mô hình PhoBERT

In [ ]:
# Tải mô hình PhoBERT với một lớp phân loại ở trên (3 nhãn)
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base",
    num_labels=3,
    id2label=id_to_label,
    label2id=label_map
)

# Đưa mô hình lên GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

## Định nghĩa tham số huấn luyện

In [ ]:
training_args = TrainingArguments(
    output_dir="./phobert_sentiment",    # thư mục lưu kết quả
    do_train=True,
    do_eval=True,
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.1,                    # sử dụng weight decay để regularization
    warmup_ratio=0.1,                    # ổn định quá trình huấn luyện, cải thiện kết quả
    eval_strategy="epoch",               # đánh giá trên val mỗi epoch
    save_strategy="epoch",               # lưu mô hình mỗi epoch
    load_best_model_at_end=True,         # tự động load mô hình tốt nhất sau train
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,              # chọn mô hình có accuracy cao nhất
    logging_dir="./train_logs",          # thư mục logging (tùy chọn)
    logging_steps=50,                    # tần suất log (nếu muốn log chi tiết hơn)
)

# Hàm tính toán metric (độ chính xác)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Callback để dừng sớm nếu mô hình không cải thiện
early_stopping = EarlyStoppingCallback(early_stopping_patience=2) # Dừng nếu sau 2 epoch không cải thiện

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
print("Bắt đầu huấn luyện...")
trainer.train() # train từ đầu
# nếu đang train mà bị crash thì dùng cái này để train tiếp từ epoch đang train
# trainer.train(resume_from_checkpoint=True)
print("Hoàn thành huấn luyện")

Bắt đầu huấn luyện...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.826800,0.717421,0.709308
2,0.784300,0.686969,0.720770
3,0.698500,0.684971,0.726272


## Lưu mô hình tốt nhất vào drive

In [ ]:
final_model_path_on_drive = "/content/drive/MyDrive/XayDungMoHinh/phobert_sentiment_model_final"

# Lưu mô hình
print(f"Bắt đầu lưu mô hình tốt nhất vào: {final_model_path_on_drive}")
trainer.save_model(final_model_path_on_drive)

# Lưu cả tokenizer vào cùng thư mục đó
tokenizer.save_pretrained(final_model_path_on_drive)
print("Đã lưu thành công mô hình và tokenizer vào Google Drive!")

Bắt đầu lưu mô hình tốt nhất vào: /content/drive/MyDrive/XayDungMoHinh/phobert_sentiment_model_final
Đã lưu thành công mô hình và tokenizer vào Google Drive!


## Tải mô hình đã lưu từ drive lên colab

In [ ]:
# TẢI LẠI MÔ HÌNH TỪ DRIVE VÀ ĐÁNH GIÁ
# Chỉ định đường dẫn tới mô hình đã lưu
saved_model_path = "/content/drive/MyDrive/XayDungMoHinh/phobert_sentiment_model_final"

print(f"Đang tải mô hình từ: {saved_model_path}")
model_loaded = AutoModelForSequenceClassification.from_pretrained(saved_model_path)
tokenizer_loaded = AutoTokenizer.from_pretrained(saved_model_path)
print("Đã tải mô hình xong")

## Đánh giá trên tập test
### Kết quả
```

Classification Report trên tập Test:\n
              precision    recall  f1-score   support

    Tiêu cực       0.89      0.81      0.85        21
   Trung lập       0.68      0.59      0.63        22
    Tích cực       0.63      0.77      0.69        22

    accuracy                           0.72        65
   macro avg       0.74      0.72      0.73        65
weighted avg       0.73      0.72      0.72        65

Accuracy trên tập Test: 0.7231
```